In [12]:
from ase.io import read,write
gulp_structure=read('gulp.xyz',format='xyz')
atoms=read('../optimized.vasp',format='vasp')
gulp_structure.set_cell(atoms.cell)

write('gulp.dat',gulp_structure,format='lammps-data')


In [13]:
natom=512
nmodes=natom*3

import numpy as np


In [14]:
dyn=np.loadtxt('Dyn.form').reshape((nmodes,nmodes))

In [15]:
dyn[0]

array([ 0.58531107, -0.01759416, -0.04394639, ...,  0.        ,
        0.        ,  0.        ])

In [16]:
from ase.io import read

atoms=read('../optimized.vasp',format='vasp')


In [17]:
masses=atoms.get_masses()


In [18]:
dyn[0:3,0:3]*masses[0]

array([[16.4384614 , -0.49413198, -1.23423436],
       [-0.49413198, 16.75081575, -1.31319478],
       [-1.23423436, -1.31319478, 15.1847998 ]])

In [22]:
dyn[3:6,3:6]*masses[0]

array([[18.50484224, -0.35696175, -1.29725823],
       [-0.35696175, 15.33447207, -1.50714923],
       [-1.29725823, -1.50714923, 14.6965109 ]])

In [20]:
#reading phonopy format force constant
FC_file='/home/emi/GULP/thermal_conductivity/aSi512_GULP/FORCE_CONSTANTS_2ND'
with open(FC_file,'r') as fc:
    lines=fc.readlines()

    nlineblock=4
    fc_all=np.zeros((natom,natom,3,3))
    start=1
    for i in range(natom):
        for j in range(natom):
            fc_block=lines[start+1:start+nlineblock]
            fc=np.loadtxt(fc_block)
            #fc_all[i,j]=fc/np.sqrt(masses[i]*masses[j])
            fc_all[i,j]=fc
            start=start+nlineblock

In [21]:
fc_all[0,0]

array([[16.43823915, -0.49410715, -1.23422592],
       [-0.49410715, 16.7505942 , -1.31317768],
       [-1.23422592, -1.31317768, 15.18455583]])

In [23]:
fc_all[1,1]

array([[18.5043507 , -0.35710017, -1.29724485],
       [-0.35710017, 15.33418511, -1.50717439],
       [-1.29724485, -1.50717439, 14.6962958 ]])

以下のはまだテストしてない

In [38]:

'''
evaluate velocity operator.
structure file: unitcell structure with vasp POSCAR format
Dyn: Flat format (low:0x,0y,0z....., column:0x,0y,0z....) natom*3xnatom*3 Dynamical matrix
(already scaled by mass, regular output of lammps dynamical_matrix) 
'''
import sys
sys.path.append('../../src')
def get_Vij_from_flat(structure_file,Dyn):
    atoms=read(structure_file,format='vasp')
    natom=len(atoms.positions)

    dist=np.zeros((natom,natom,3))

    from src.nearest import find_nearest
    for i in range(natom):  
        for j in range(i):
            xdc,ydc,zdc,rmin=find_nearest(atoms,i,j)
            dist[i,j]=np.array([xdc,ydc,zdc])
            #invert
            dist[j,i]=-np.array([xdc,ydc,zdc])
    
    
    Rx=np.repeat(dist[:,:,0],3,axis=1)
    Rx=np.repeat(Rx,3,axis=0)
    Ry=np.repeat(dist[:,:,1],3,axis=1)
    Ry=np.repeat(Ry,3,axis=0)
    Rz=np.repeat(dist[:,:,2],3,axis=1)
    Rz=np.repeat(Rz,3,axis=0)  

    Vx=np.zeros((natom*3,natom*3))
    Vy=np.zeros((natom*3,natom*3))
    Vz=np.zeros((natom*3,natom*3))

    '''
    triu=np.triu(np.ones((natom*3,natom*3)))
    tril=np.tril(np.ones((natom*3,natom*3)),-1)
    mask=triu+tril*-1
    '''

    Vx=np.dot(Rx,Dyn)
    Vy=np.dot(Ry,Dyn)
    Vx=np.dot(Rz,Dyn)

    return Vx, Vy, Vz

In [29]:
import sys
sys.path.append('../../src')

In [30]:
from nearest import find_nearest
dist=np.zeros((natom,natom,3))
for i in range(natom):  
    for j in range(i):
        xdc,ydc,zdc,rmin=find_nearest(atoms,i,j)
        dist[i,j]=np.array([xdc,ydc,zdc])
        #invert
        dist[j,i]=-np.array([xdc,ydc,zdc])

In [31]:
Rx=np.repeat(dist[:,:,0],3,axis=1)
Rx=np.repeat(Rx,3,axis=0)
Ry=np.repeat(dist[:,:,1],3,axis=1)
Ry=np.repeat(Ry,3,axis=0)
Rz=np.repeat(dist[:,:,2],3,axis=1)
Rz=np.repeat(Rz,3,axis=0)  

In [32]:
Rx.shape

(1536, 1536)

In [35]:
Rx[0,0:6]

array([  0.        ,   0.        ,   0.        , -10.14206505,
       -10.14206505, -10.14206505])

In [36]:
Rx[1,0:6]

array([  0.        ,   0.        ,   0.        , -10.14206505,
       -10.14206505, -10.14206505])

In [37]:
Rx[3,0:6]

array([10.14206505, 10.14206505, 10.14206505,  0.        ,  0.        ,
        0.        ])